<a href="https://colab.research.google.com/github/phanee16/Solar-Power-Estimator/blob/main/prepare_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Read PV data


In [2]:
import requests
import numpy as np
import pandas as pd
def load_data(url, sep=','):
    df = pd.read_csv(url, sep=sep, parse_dates=['timestamp'], index_col='timestamp')
    return df

url = 'https://raw.githubusercontent.com/phanee16/Solar-Power-Estimator/main/SunPower_Historical.csv'

#read dataset
PV_data = load_data(url, ',')


PV_data

,Active_Power
timestamp,
2013-07-22 00:00:00,0.0
2013-07-22 00:05:00,0.0
2013-07-22 00:10:00,0.0
2013-07-22 00:15:00,0.0
2013-07-22 00:20:00,0.0
...,...
2023-05-27 23:35:00,0.0
2023-05-27 23:40:00,0.0
2023-05-27 23:45:00,0.0


In [3]:
#extract hourly data
PV_data = PV_data[PV_data.index.minute == 0]
PV_data

,Active_Power
timestamp,
2013-07-22 00:00:00,0.0
2013-07-22 01:00:00,0.0
2013-07-22 02:00:00,0.0
2013-07-22 03:00:00,0.0
2013-07-22 04:00:00,0.0
...,...
2023-05-27 19:00:00,0.0
2023-05-27 20:00:00,0.0
2023-05-27 21:00:00,NaN


Extract Weather Historical Data from OpenMeteo API

In [5]:
Predictors= ['temperature_2m', 'relativehumidity_2m', 'direct_radiation', 'diffuse_radiation',  'windspeed_10m', 'cloudcover']


weather_df = pd.DataFrame(columns = Predictors)

#latitude and longitude of the panel
lat = -23.760363
long = 133.874719

#start and end date of the available solar power data
start = str(PV_data.index[0].strftime('%Y-%m-%d'))
end = str(PV_data.index[-1].strftime('%Y-%m-%d'))

#historical weather for the panel
r = requests.get('https://archive-api.open-meteo.com/v1/archive', params={'latitude':lat, 'longitude': long, 'timezone': 'auto', 'start_date':start , 'end_date': end , 'hourly' : Predictors}).json() #timezone = auto so that it matches the local timezone

#create weather dataset
time = pd.to_datetime(np.array(r['hourly']['time']))
weather_df['timestamp'] = time

for p in Predictors:
        weather_df[p] = np.array(r['hourly'][p])

#set timestamp index
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])
weather_df.set_index('timestamp', inplace=True)

weather_df.head()

,temperature_2m,relativehumidity_2m,direct_radiation,diffuse_radiation,windspeed_10m,cloudcover
timestamp,,,,,,
2013-07-22 00:00:00,6.7,67,0.0,0.0,10.2,0
2013-07-22 01:00:00,5.7,71,0.0,0.0,9.2,0
2013-07-22 02:00:00,4.8,76,0.0,0.0,9.1,0
2013-07-22 03:00:00,4.1,79,0.0,0.0,10.2,0
2013-07-22 04:00:00,3.4,81,0.0,0.0,10.2,0


Merge datasets and export

In [7]:
#merge datasets
full_df_weather = PV_data.merge(weather_df, how = 'left', left_on = PV_data.index, right_index= True)

print(full_df_weather.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 85384 entries, 2013-07-22 00:00:00 to 2023-05-27 23:00:00
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Active_Power         83628 non-null  float64
 1   temperature_2m       85384 non-null  float64
 2   relativehumidity_2m  85384 non-null  int64  
 3   direct_radiation     85384 non-null  float64
 4   diffuse_radiation    85384 non-null  float64
 5   windspeed_10m        85384 non-null  float64
 6   cloudcover           85384 non-null  int64  
dtypes: float64(5), int64(2)
memory usage: 5.2 MB
None


In [8]:
full_df_weather.to_csv('SunPower_Merged.csv', sep = '\t')